This program assigns roles to team members for the project. 

Conditions:
1. Team will rotate team roles in "cycles".
2. The length of each cycle may be two weeks or more, as determined by the team.
3. Each role has an expert who will guide the team members assigned to the role. 
4. Each team member may be assigned to more than one role at any given time. 
5. No team members, besides the leads, will be assigned to the same role in two consecutive cycles. 
6. Experts may be assigned to other roles just like any other team member. 

In [85]:
import pandas as pd
import numpy as np
import random
random.seed(42)

# Assign Roles 

This part of the code is used to assign roles to the team members from scratch

Instantiate list of members

In [86]:
members = [
            "Oluwaseyi Adaramola",
            "Farook Ajose",
            "Malik Akintola",
            "Ibukun Akinleye",
            "Samson Ilesanmi",
            "Oluwaseunfunmi Alarima",
            "Damilola Omokehinde",
            "Samuel Olayiwola",
            "Adejuwon Adegite",
            "Kafilat Akinyemi",
            "Chibike Enibe",
            "Ayodeji Abatan",
            "Divine Nkwocha",
            "AbdulQahhar Okenla",
        ]

members

['Oluwaseyi Adaramola',
 'Farook Ajose',
 'Malik Akintola',
 'Ibukun Akinleye',
 'Samson Ilesanmi',
 'Oluwaseunfunmi Alarima',
 'Damilola Omokehinde',
 'Samuel Olayiwola',
 'Adejuwon Adegite',
 'Kafilat Akinyemi',
 'Chibike Enibe',
 'Ayodeji Abatan',
 'Divine Nkwocha',
 'AbdulQahhar Okenla']

Instantiate available roles

In [87]:
roles = pd.DataFrame({
    "role": ["Project Manager",
            "Data Analyst (Python)",
            "Data Scientist/Statistician",
            "Visualization Specialist (Power BI)",
            "Technical Writer",
            "Publicity Coordinator",
            "Quality Assurance Engineer"],
    "num_people": [2, 3, 3, 3, 3, 3, 3],
    "lead": [["Farook Ajose", "Oluwaseyi Adaramola"],
               "Malik Akintola",
               "Ayodeji Abatan",
               "Samson Ilesanmi",
               "Ibukun Akinleye",
               "AbdulQahhar Okenla",
               "Farook Ajose"]
    })
roles

,role,num_people,lead
0,Project Manager,2,"[Farook Ajose, Oluwaseyi Adaramola]"
1,Data Analyst (Python),3,Malik Akintola
2,Data Scientist/Statistician,3,Ayodeji Abatan
3,Visualization Specialist (Power BI),3,Samson Ilesanmi
4,Technical Writer,3,Ibukun Akinleye
5,Publicity Coordinator,3,AbdulQahhar Okenla
6,Quality Assurance Engineer,3,Farook Ajose


Write function to assign roles

In [88]:
num_cycles = 6

def assign_roles(row, cycles=num_cycles, team=members):
    """Assign team members to different roles for each rotation cycle, ensuring that an expert in the area is present to guide them. 

    Args:
        row (pandas.Series): A row of the `roles` dataframe containing the role details.
        cycles (int, optional): Number of rotation cycles to assign the roles for. Defaults to the value of `num_cycles`.
        team (list, optional): A list of team members to select from. Defaults to the value of `members`.

    Returns:
        lead_cycle_dict (dict): A dictionary containing the assigned members for each cycle, with keys in the format "Cycle x" where x is the cycle number.
    """
    
    # Remove expert from sample list 
    lead = row['lead']
    no_lead_team = team.copy()
    if type(lead) == list:
        for item in lead:
            no_lead_team.remove(item)
    else:
        no_lead_team.remove(lead)

    # Create dictionary of assigned members for each cycle
    cycle_dict = {
        f"Cycle {cycle}":  np.random.choice(no_lead_team, size=row["num_people"]-1, replace=False)
        for cycle in range(1, cycles+1)
    }
    
    # Remove common values between consecutive cycles
    for i in range(1, cycles):
        current_cycle = cycle_dict[f"Cycle {i}"]
        next_cycle = cycle_dict[f"Cycle {i+1}"]
        common_members = np.intersect1d(current_cycle, next_cycle)
        for member in common_members:
            new_member = np.random.choice(no_lead_team, size=1)[0]
            while new_member in next_cycle:
                new_member = np.random.choice(no_lead_team, size=1)[0]
            next_cycle = np.where(next_cycle == member, new_member, next_cycle)
        cycle_dict[f"Cycle {i+1}"] = next_cycle
        
    # Append expert to list and convert to string    
    lead_cycle_dict = {
        f"Cycle {cycle}":  ", ".join(np.append(lead, cycle_dict[f"Cycle {cycle}"]))
        for cycle in range(1, cycles+1)
    }

    return lead_cycle_dict
        
# Test function to see outputs.     
assign_roles(roles.iloc[2, :])

{'Cycle 1': 'Ayodeji Abatan, Samuel Olayiwola, Adejuwon Adegite',
 'Cycle 2': 'Ayodeji Abatan, Chibike Enibe, Ibukun Akinleye',
 'Cycle 3': 'Ayodeji Abatan, Adejuwon Adegite, Kafilat Akinyemi',
 'Cycle 4': 'Ayodeji Abatan, Samuel Olayiwola, Farook Ajose',
 'Cycle 5': 'Ayodeji Abatan, Ibukun Akinleye, Chibike Enibe',
 'Cycle 6': 'Ayodeji Abatan, Damilola Omokehinde, Adejuwon Adegite'}

In [89]:
type([1, 2]) == list

True

Update dataframe with function

In [90]:
roles[[f"Cycle {cycle}" for cycle in range(1, num_cycles+1)]] = roles.apply(func=assign_roles, axis=1, result_type='expand')
roles

,role,num_people,lead,Cycle 1,Cycle 2,Cycle 3,Cycle 4,Cycle 5,Cycle 6
0,Project Manager,2,"[Farook Ajose, Oluwaseyi Adaramola]","Farook Ajose, Oluwaseyi Adaramola, Divine Nkwocha","Farook Ajose, Oluwaseyi Adaramola, Malik Akintola","Farook Ajose, Oluwaseyi Adaramola, Oluwaseunfu...","Farook Ajose, Oluwaseyi Adaramola, Ibukun Akin...","Farook Ajose, Oluwaseyi Adaramola, Kafilat Aki...","Farook Ajose, Oluwaseyi Adaramola, Oluwaseunfu..."
1,Data Analyst (Python),3,Malik Akintola,"Malik Akintola, Kafilat Akinyemi, Ayodeji Abatan","Malik Akintola, Oluwaseunfunmi Alarima, Adejuw...","Malik Akintola, Ibukun Akinleye, Divine Nkwocha","Malik Akintola, Damilola Omokehinde, AbdulQahh...","Malik Akintola, Kafilat Akinyemi, Divine Nkwocha","Malik Akintola, Samson Ilesanmi, Samuel Olayiwola"
2,Data Scientist/Statistician,3,Ayodeji Abatan,"Ayodeji Abatan, Farook Ajose, Divine Nkwocha","Ayodeji Abatan, Chibike Enibe, Kafilat Akinyemi","Ayodeji Abatan, Ibukun Akinleye, AbdulQahhar O...","Ayodeji Abatan, Chibike Enibe, Damilola Omokeh...","Ayodeji Abatan, Malik Akintola, Divine Nkwocha","Ayodeji Abatan, Adejuwon Adegite, AbdulQahhar ..."
3,Visualization Specialist (Power BI),3,Samson Ilesanmi,"Samson Ilesanmi, Damilola Omokehinde, Malik Ak...","Samson Ilesanmi, Samuel Olayiwola, Adejuwon Ad...","Samson Ilesanmi, Chibike Enibe, Ibukun Akinleye","Samson Ilesanmi, Samuel Olayiwola, Divine Nkwocha","Samson Ilesanmi, Ibukun Akinleye, Farook Ajose","Samson Ilesanmi, Adejuwon Adegite, Divine Nkwocha"
4,Technical Writer,3,Ibukun Akinleye,"Ibukun Akinleye, Oluwaseunfunmi Alarima, Damil...","Ibukun Akinleye, AbdulQahhar Okenla, Ayodeji A...","Ibukun Akinleye, Damilola Omokehinde, Samuel O...","Ibukun Akinleye, Ayodeji Abatan, Malik Akintola","Ibukun Akinleye, Adejuwon Adegite, Oluwaseyi A...","Ibukun Akinleye, Farook Ajose, Chibike Enibe"
5,Publicity Coordinator,3,AbdulQahhar Okenla,"AbdulQahhar Okenla, Chibike Enibe, Malik Akintola","AbdulQahhar Okenla, Damilola Omokehinde, Faroo...","AbdulQahhar Okenla, Kafilat Akinyemi, Chibike ...","AbdulQahhar Okenla, Oluwaseyi Adaramola, Samue...","AbdulQahhar Okenla, Samson Ilesanmi, Adejuwon ...","AbdulQahhar Okenla, Ayodeji Abatan, Divine Nkw..."
6,Quality Assurance Engineer,3,Farook Ajose,"Farook Ajose, Malik Akintola, Samuel Olayiwola","Farook Ajose, Chibike Enibe, Damilola Omokehinde","Farook Ajose, Ibukun Akinleye, AbdulQahhar Okenla","Farook Ajose, Divine Nkwocha, Chibike Enibe","Farook Ajose, Adejuwon Adegite, Oluwaseunfunmi...","Farook Ajose, Divine Nkwocha, AbdulQahhar Okenla"


Summarize the roles for each member per cycle

In [91]:
def summarize_roles(df, members=members, cycles=num_cycles):
    role_summary = {}
    for member in members:
        member_roles = []
        for cycle in range(1, cycles+1):
            assigned_roles = ", ".join(df["role"][df[f"Cycle {cycle}"].str.contains(member)])
            member_roles.append(assigned_roles)
        role_summary[member] = member_roles
        
    df = pd.DataFrame(role_summary).T
    df.columns = [f"Cycle {cycle}" for cycle in range(1, cycles+1)]
    
    return df

In [92]:
role_summary = summarize_roles(roles).convert_dtypes('object')
role_summary

,Cycle 1,Cycle 2,Cycle 3,Cycle 4,Cycle 5,Cycle 6
Oluwaseyi Adaramola,Project Manager,Project Manager,Project Manager,"Project Manager, Publicity Coordinator","Project Manager, Technical Writer",Project Manager
Farook Ajose,"Project Manager, Data Scientist/Statistician, ...","Project Manager, Publicity Coordinator, Qualit...","Project Manager, Quality Assurance Engineer","Project Manager, Quality Assurance Engineer","Project Manager, Visualization Specialist (Pow...","Project Manager, Technical Writer, Quality Ass..."
Malik Akintola,"Data Analyst (Python), Visualization Specialis...","Project Manager, Data Analyst (Python)",Data Analyst (Python),"Data Analyst (Python), Technical Writer","Data Analyst (Python), Data Scientist/Statisti...",Data Analyst (Python)
Ibukun Akinleye,Technical Writer,Technical Writer,"Data Analyst (Python), Data Scientist/Statisti...","Project Manager, Technical Writer","Visualization Specialist (Power BI), Technical...",Technical Writer
Samson Ilesanmi,Visualization Specialist (Power BI),Visualization Specialist (Power BI),Visualization Specialist (Power BI),Visualization Specialist (Power BI),"Visualization Specialist (Power BI), Publicity...","Data Analyst (Python), Visualization Specialis..."
Oluwaseunfunmi Alarima,Technical Writer,Data Analyst (Python),Project Manager,,Quality Assurance Engineer,Project Manager
Damilola Omokehinde,"Visualization Specialist (Power BI), Technical...","Publicity Coordinator, Quality Assurance Engineer",Technical Writer,"Data Analyst (Python), Data Scientist/Statisti...",,
Samuel Olayiwola,Quality Assurance Engineer,Visualization Specialist (Power BI),Technical Writer,"Visualization Specialist (Power BI), Publicity...",,Data Analyst (Python)
Adejuwon Adegite,,"Data Analyst (Python), Visualization Specialis...",,,"Technical Writer, Publicity Coordinator, Quali...","Data Scientist/Statistician, Visualization Spe..."
Kafilat Akinyemi,Data Analyst (Python),Data Scientist/Statistician,Publicity Coordinator,,"Project Manager, Data Analyst (Python)",


Save dataframe to CSV

In [97]:
# roles.to_csv("assigned_roles.csv", index=False)
# role_summary.to_csv("role_summary.csv")

# Update Existing Roles 

This part of the code is used to update the existing roles in a CSV.

In [94]:
current_roles = pd.read_csv("assigned_roles.csv")
current_roles.head()

,role,num_people,expert,Cycle 1,Cycle 2,Cycle 3,Cycle 4,Cycle 5,Cycle 6
0,Data Analyst (Python),4,Malik Akintola,"Malik Akintola, Chibike Enibe, AbdulQahhar Oke...","Malik Akintola, Samson Ilesanmi, Kafilat Akiny...","Malik Akintola, Adejuwon Adegite, Chibike Enib...","Malik Akintola, AbdulQahhar Okenla, Farook Ajo...","Malik Akintola, Adejuwon Adegite, Ayodeji Abat...","Malik Akintola, Farook Ajose, Chibike Enibe, D..."
1,Data Scientist/Statistician,4,Ayodeji Abatan,"Ayodeji Abatan, AbdulQahhar Okenla, Divine Nkw...","Ayodeji Abatan, Kafilat Akinyemi, Chibike Enib...","Ayodeji Abatan, Ibukun Akinleye, Oluwaseyi Ada...","Ayodeji Abatan, Oluwaseunfunmi Alarima, Samson...","Ayodeji Abatan, Samuel Olayiwola, Farook Ajose...","Ayodeji Abatan, Malik Akintola, Oluwaseyi Adar..."
2,Visualization Specialist (Power BI),4,Samson Ilesanmi,"Samson Ilesanmi, Ayodeji Abatan, Damilola Omok...","Samson Ilesanmi, Ibukun Akinleye, Farook Ajose...","Samson Ilesanmi, Samuel Olayiwola, Divine Nkwo...","Samson Ilesanmi, Ayodeji Abatan, AbdulQahhar O...","Samson Ilesanmi, Ibukun Akinleye, Oluwaseunfun...","Samson Ilesanmi, AbdulQahhar Okenla, Divine Nk..."
3,Technical Writer,4,Farook Ajose,"Farook Ajose, Oluwaseunfunmi Alarima, AbdulQah...","Farook Ajose, Samson Ilesanmi, Adejuwon Adegit...","Farook Ajose, Chibike Enibe, Samuel Olayiwola,...","Farook Ajose, Oluwaseyi Adaramola, AbdulQahhar...","Farook Ajose, Ibukun Akinleye, Chibike Enibe, ...","Farook Ajose, AbdulQahhar Okenla, Oluwaseyi Ad..."
4,Publicity Coordinator,3,AbdulQahhar Okenla,"AbdulQahhar Okenla, Ibukun Akinleye, Samuel Ol...","AbdulQahhar Okenla, Damilola Omokehinde, Adeju...","AbdulQahhar Okenla, Malik Akintola, Ibukun Aki...","AbdulQahhar Okenla, Samson Ilesanmi, Adejuwon ...","AbdulQahhar Okenla, Ibukun Akinleye, Kafilat A...","AbdulQahhar Okenla, Oluwaseyi Adaramola, Divin..."


Summarize existing roles

In [95]:
current_role_summary = summarize_roles(current_roles)
current_role_summary

,Cycle 1,Cycle 2,Cycle 3,Cycle 4,Cycle 5,Cycle 6
Oluwaseyi Adaramola,Data Analyst (Python),,"Data Scientist/Statistician, Visualization Spe...",Technical Writer,,"Data Scientist/Statistician, Technical Writer,..."
Farook Ajose,Technical Writer,"Data Scientist/Statistician, Visualization Spe...",Technical Writer,"Data Analyst (Python), Technical Writer","Data Scientist/Statistician, Technical Writer","Data Analyst (Python), Technical Writer"
Malik Akintola,"Data Analyst (Python), Data Scientist/Statisti...",Data Analyst (Python),"Data Analyst (Python), Publicity Coordinator","Data Analyst (Python), Visualization Specialis...",Data Analyst (Python),"Data Analyst (Python), Data Scientist/Statisti..."
Ibukun Akinleye,Publicity Coordinator,"Data Analyst (Python), Visualization Specialis...","Data Scientist/Statistician, Technical Writer,...",,"Visualization Specialist (Power BI), Technical...",
Samson Ilesanmi,Visualization Specialist (Power BI),"Data Analyst (Python), Visualization Specialis...",Visualization Specialist (Power BI),"Data Scientist/Statistician, Visualization Spe...","Visualization Specialist (Power BI), Technical...",Visualization Specialist (Power BI)
Oluwaseunfunmi Alarima,Technical Writer,,,Data Scientist/Statistician,Visualization Specialist (Power BI),
Damilola Omokehinde,Visualization Specialist (Power BI),Publicity Coordinator,Data Scientist/Statistician,Data Analyst (Python),Visualization Specialist (Power BI),Data Analyst (Python)
Samuel Olayiwola,Publicity Coordinator,,"Data Analyst (Python), Visualization Specialis...",,Data Scientist/Statistician,
Adejuwon Adegite,,"Technical Writer, Publicity Coordinator",Data Analyst (Python),Publicity Coordinator,Data Analyst (Python),
Kafilat Akinyemi,Visualization Specialist (Power BI),"Data Analyst (Python), Data Scientist/Statisti...",,,"Data Analyst (Python), Publicity Coordinator",Data Scientist/Statistician


Save existing role summary

In [96]:
# current_role_summary.to_csv("role_summary.csv")